#Relatório do Processo de ETL com Dados da RAIS


### Introdução

Este relatório detalha as etapas seguidas, as dificuldades encontradas e as correções aplicadas durante o processo de Extração, Transformação e Carregamento (ETL) de dados da Relação Anual de Informações Sociais (RAIS) para a Região Norte do Brasil, referentes ao ano de 2021. O objetivo principal foi preparar os dados para análises futuras, especificamente para identificar as 5 atividades e os 5 municípios que mais empregam na região.

### Extração dos Dados

O processo começou com a extração dos dados necessários do servidor FTP do Ministério do Trabalho e Emprego. Utilizei o FileZilla para acessar o servidor e baixar o arquivo da RAIS da Região Norte de 2021, além do arquivo de layout para a interpretação correta dos dados.

### Tratamento dos Dados

Com os arquivos baixados, procedi ao tratamento dos dados utilizando o Pandas em um Jupyter Notebook no Google Colab. Após carregar os dados do arquivo `RAIS_VINC_PUB_NORTE.txt`, realizei as seguintes operações:

1. **Filtragem dos Dados**: Removi os registros onde a variável "EMP EM 31/12" era igual a 0, indicando pessoas que não estavam empregadas no final do ano. Contudo, enfrentei um problema ao aplicar este filtro, devido a um erro de `KeyError`, que se revelou ser um equívoco na identificação da coluna correta. A coluna correta era "Vínculo Ativo 31/12".

2. **Criação da Nova Variável**: Tentei criar a variável "DIV CNAE 20" a partir dos dois primeiros dígitos da "CLAS CNAE 20". No entanto, houve um erro na aplicação desta operação, decorrente de uma confusão nas colunas utilizadas.

### Carregamento dos Dados

Antes de realizar o carregamento dos dados para um banco de dados MySQL, iniciei a modelagem da tabela `dados_rais` no MySQL Workbench. Aqui, defini as colunas com base nos dados disponíveis e seus tipos, mas enfrentei dificuldades em ajustar corretamente os tipos de dados e as dimensões das colunas `VARCHAR`, além de identificar as colunas que deveriam permitir valores nulos.

### Erros e Correções

- **Erro de `KeyError` na Filtragem dos Dados**: O erro foi causado por uma referência incorreta ao nome da coluna. A correção envolveu a utilização do nome correto da coluna, "Vínculo Ativo 31/12".

- **Problema na Criação da Variável "DIV CNAE 20"**: A dificuldade na criação desta variável foi superada após corrigir a referência à coluna correta e garantir a correta transformação dos dados para string antes de aplicar a extração dos dois primeiros dígitos.

- **Modelagem da Tabela no Banco de Dados**: Ajustes foram necessários para alinhar os tipos de dados e tamanhos das colunas `VARCHAR` com os dados processados, além de considerar a utilização de `DECIMAL` em vez de `FLOAT` para valores monetários, visando uma maior precisão.

### Conclusão

Apesar dos desafios encontrados, o processo de ETL proporcionou uma valiosa experiência de aprendizado em manipulação, tratamento e preparação de dados para análise. As dificuldades enfrentadas destacaram a importância de uma verificação cuidadosa dos nomes das colunas, a precisão na definição dos tipos de dados e a necessidade de uma análise exploratória para guiar as decisões de modelagem de dados. Ao reiniciar o processo em outro notebook, apliquei as correções necessárias, alcançando os resultados esperados e preparando o terreno para as análises subsequentes.

#Processo de ETL com Dados da RAIS

In [ ]:
import pandas as pd

caminho_do_arquivo = '/content/RAIS_VINC_PUB_NORTE.txt'

try:
    dados = pd.read_csv(caminho_do_arquivo, sep=';', encoding='latin1', low_memory=False)
    print('Arquivo lido com sucesso!')
except Exception as e:
    print(f'Erro ao ler o arquivo: {e}')

Arquivo lido com sucesso!


In [ ]:
print(dados.columns)

Index(['Bairros SP', 'Bairros Fortaleza', 'Bairros RJ', 'Causa Afastamento 1',
       'Causa Afastamento 2', 'Causa Afastamento 3', 'Motivo Desligamento',
       'CBO Ocupação 2002', 'CNAE 2.0 Classe', 'CNAE 95 Classe',
       'Distritos SP', 'Vínculo Ativo 31/12', 'Faixa Etária',
       'Faixa Hora Contrat', 'Faixa Remun Dezem (SM)',
       'Faixa Remun Média (SM)', 'Faixa Tempo Emprego',
       'Escolaridade após 2005', 'Qtd Hora Contr', 'Idade',
       'Ind CEI Vinculado', 'Ind Simples', 'Mês Admissão', 'Mês Desligamento',
       'Mun Trab', 'Município', 'Nacionalidade', 'Natureza Jurídica',
       'Ind Portador Defic', 'Qtd Dias Afastamento', 'Raça Cor',
       'Regiões Adm DF', 'Vl Remun Dezembro Nom', 'Vl Remun Dezembro (SM)',
       'Vl Remun Média Nom', 'Vl Remun Média (SM)', 'CNAE 2.0 Subclasse',
       'Sexo Trabalhador', 'Tamanho Estabelecimento', 'Tempo Emprego',
       'Tipo Admissão', 'Tipo Estab', 'Tipo Estab.1', 'Tipo Defic',
       'Tipo Vínculo', 'IBGE Subsetor', 'Vl 

In [ ]:
dados_filtrados = dados[dados['Vínculo Ativo 31/12'] != 0]
print(f'Registros antes da filtragem: {len(dados)}, após filtragem: {len(dados_filtrados)}')

Registros antes da filtragem: 21486, após filtragem: 1


In [ ]:
dados_filtrados['DIV CNAE 20'] = dados_filtrados['CNAE 2.0 Classe'].astype(str).apply(lambda x: x[:2] if pd.notnull(x) else x)
print('Nova variável criada com sucesso!')

Nova variável criada com sucesso!


<ipython-input-4-0e09a115135c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_filtrados['DIV CNAE 20'] = dados_filtrados['CNAE 2.0 Classe'].astype(str).apply(lambda x: x[:2] if pd.notnull(x) else x)


In [ ]:
print(dados_filtrados.dtypes)

Bairros SP                object
Bairros Fortaleza         object
Bairros RJ                object
Causa Afastamento 1        int64
Causa Afastamento 2        int64
                          ...   
Vl Rem Novembro SC        object
Ano Chegada Brasil       float64
Ind Trab Intermitente    float64
Ind Trab Parcial         float64
DIV CNAE 20               object
Length: 61, dtype: object


In [ ]:
max_length_bairros_sp = dados_filtrados['Bairros SP'].astype(str).map(len).max()
max_length_bairros_fortaleza = dados_filtrados['Bairros Fortaleza'].astype(str).map(len).max()
max_length_bairros_rj = dados_filtrados['Bairros RJ'].astype(str).map(len).max()
max_length_div_cnae_20 = dados_filtrados['DIV CNAE 20'].astype(str).map(len).max()

print(f"Comprimento máximo para Bairros SP: {max_length_bairros_sp}")
print(f"Comprimento máximo para Bairros Fortaleza: {max_length_bairros_fortaleza}")
print(f"Comprimento máximo para Bairros RJ: {max_length_bairros_rj}")
print(f"Comprimento máximo para DIV CNAE 20: {max_length_div_cnae_20}")

Comprimento máximo para Bairros SP: 20
Comprimento máximo para Bairros Fortaleza: 20
Comprimento máximo para Bairros RJ: 20
Comprimento máximo para DIV CNAE 20: 2


# Conectando ao Banco

In [ ]:
!pip install sqlalchemy pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 958.3 kB/s eta 0:00:00


In [ ]:
from sqlalchemy import create_engine

usuario = 'root '
senha = '18031999'
host = '127.0.0.1:3306'
database = 'ProcessoSeletivoDataViva'

engine = create_engine(f'mysql+pymysql://{usuario}:{senha}@{host}/{database}')

dados_filtrados.to_sql('dados_rais', con=engine, index=False, if_exists='append', chunksize=500)

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 111] Connection refused)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)